# Bibliotecas

In [14]:
import os
import re

import requests
from bs4 import BeautifulSoup

from pprint import pprint
from tqdm.notebook import tqdm as pbar

# Variáveis

In [15]:
url_base = 'https://www.sbc.org.br'
url_provas = f'{url_base}/documentos-da-sbc/category/153-provas-e-gabaritos-do-poscomp'
url_editais = f'{url_base}/documentos-da-sbc/category/186-editais'
url_outros = f'{url_base}/documentos-da-sbc/category/185-poscomp'

# Download

## Provas e Gabaritos

In [16]:
res = requests.get(url_provas)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [17]:
links = {}

for link in soup.find_all('a'):  # todos os links
  if link.get('href').find('category') >= 0 and not link.find('img') and link.text != 'Up':
    # 'category' na url, não é uma img, o texto não é 'Up' (link para a categoria superior)
    links[link.get_text()] = url_base + link.get('href')

pprint(links)

{'2002': 'https://www.sbc.org.br/documentos-da-sbc/category/158-2002',
 '2003': 'https://www.sbc.org.br/documentos-da-sbc/category/162-2003',
 '2004': 'https://www.sbc.org.br/documentos-da-sbc/category/156-2004',
 '2005': 'https://www.sbc.org.br/documentos-da-sbc/category/161-2005',
 '2006': 'https://www.sbc.org.br/documentos-da-sbc/category/159-2006',
 '2007': 'https://www.sbc.org.br/documentos-da-sbc/category/163-2007',
 '2008': 'https://www.sbc.org.br/documentos-da-sbc/category/154-2008',
 '2009': 'https://www.sbc.org.br/documentos-da-sbc/category/155-2009',
 '2010': 'https://www.sbc.org.br/documentos-da-sbc/category/160-2010',
 '2011': 'https://www.sbc.org.br/documentos-da-sbc/category/157-2011',
 '2012': 'https://www.sbc.org.br/documentos-da-sbc/category/179-2012',
 '2013': 'https://www.sbc.org.br/documentos-da-sbc/category/180-2013',
 '2014': 'https://www.sbc.org.br/documentos-da-sbc/category/181-2014',
 '2015': 'https://www.sbc.org.br/documentos-da-sbc/category/192-2015',
 '2016

In [18]:
def get_links_arquivos(link):
  if 'title' in link.attrs and link.attrs['title'] == 'Download' and link.get_text() != 'Download':
    return True
  return False

def get_tipo_arquivo(link_arquivo, links_pagina):
  links_relacionados = filter(
    lambda link: link.get('href') == link_arquivo.get('href'),
    links_pagina
  )
  for link in links_relacionados:
    img = link.find_all('img')
    if img and 'alt' in img[0].attrs:
      return img[0].attrs['alt'].split('.')[0]

In [19]:
for ano in pbar(links.keys()):
  url = links[ano]
  res = requests.get(url)
  html = res.text
  soup = BeautifulSoup(html, 'html.parser')

  links_pagina = soup.find_all('a')
  links_arquivos = list(filter(
    get_links_arquivos,
    links_pagina,
  ))

  for link in links_arquivos:
    url_arquivo = link.get('href')
    
    nome_arquivo = link.get_text()
    # tipo_arquivo = get_tipo_arquivo(link, links_pagina)
    tipo_arquivo = 'pdf'
    # todos os arquivos são pdf apesar de alguns estarem com ícone de zip

    caminho_pasta = os.path.join('..', 'arquivos', 'provas-e-gabaritos', ano)
    os.makedirs(caminho_pasta, exist_ok=True)
    caminho_arquivo = f'{caminho_pasta}/{nome_arquivo}.{tipo_arquivo}'

    if not os.path.isfile(caminho_arquivo):
      arquivo = requests.get(url_base + url_arquivo.replace('summary', 'send'))
      with open(caminho_arquivo, 'wb+') as f:
        f.write(arquivo.content)

  0%|          | 0/19 [00:00<?, ?it/s]

## Editais

In [20]:
res = requests.get(url_editais)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [21]:
links_pagina = soup.find_all('a')
links_arquivos = list(filter(
  get_links_arquivos,
  links_pagina
))

for link in pbar(links_arquivos):
  url_arquivo = link.get('href')

  nome_arquivo = link.get_text()
  tipo_arquivo = 'pdf'

  caminho_pasta = os.path.join('..', 'arquivos', 'editais')
  os.makedirs(caminho_pasta, exist_ok=True)
  caminho_arquivo = f'{caminho_pasta}/{nome_arquivo}.{tipo_arquivo}'

  if not os.path.isfile(caminho_arquivo):
    arquivo = requests.get(url_base + url_arquivo.replace('summary', 'send'))
    with open(caminho_arquivo, 'wb+') as f:
      f.write(arquivo.content)

  0%|          | 0/8 [00:00<?, ?it/s]

## Outros

In [22]:
res = requests.get(url_outros)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

In [23]:
links_pagina = soup.find_all('a')
links_arquivos = list(filter(
  get_links_arquivos,
  links_pagina
))

for link in pbar(links_arquivos):
  url_arquivo = link.get('href')

  nome_arquivo = link.get_text()
  tipo_arquivo = 'pdf'
  
  caminho_pasta = os.path.join('..', 'arquivos', 'outros')
  os.makedirs(caminho_pasta, exist_ok=True)
  caminho_arquivo = f'{caminho_pasta}/{nome_arquivo}.{tipo_arquivo}'

  if not os.path.isfile(caminho_arquivo):
    arquivo = requests.get(url_base + url_arquivo.replace('summary', 'send'))
    with open(caminho_arquivo, 'wb+') as f:
      f.write(arquivo.content)

  0%|          | 0/2 [00:00<?, ?it/s]